In [1]:
import pandas as pd
import numpy as np
import openai

In [25]:
from dotenv import load_dotenv
import os
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
data = pd.read_csv('Corona_NLP_train.csv', encoding='latin1')

In [4]:
data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [27]:
data = data[['OriginalTweet', 'Sentiment']][:30]

In [28]:
data.shape

(30, 2)

In [29]:
data.head()

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [30]:
data['Sentiment'].value_counts()

Positive              10
Neutral                6
Extremely Positive     6
Extremely Negative     4
Negative               4
Name: Sentiment, dtype: int64

In [31]:
def get_embedding(text, model="text-similarity-davinci-001"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']
 

In [32]:
data['babbage_similarity'] = data.OriginalTweet.apply(lambda x: get_embedding(x, model='text-similarity-babbage-001'))

In [34]:
data.to_csv('embedded_30_tweets.csv', index=False)

In [35]:
df = pd.read_csv('embedded_30_tweets.csv')
df.head()

,OriginalTweet,Sentiment,babbage_similarity
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,"[-0.006444347091019154, 0.013177386485040188, ..."
1,advice Talk to your neighbours family to excha...,Positive,"[-0.006633083801716566, 0.01087373960763216, -..."
2,Coronavirus Australia: Woolworths to give elde...,Positive,"[-0.005033580120652914, 0.01030619814991951, -..."
3,My food stock is not the only one which is emp...,Positive,"[-0.01348057109862566, 0.019621720537543297, 0..."
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative,"[-0.026726385578513145, 0.0207790806889534, 0...."


In [2]:
df.shape

NameError: name 'df' is not defined

In [36]:
df['babbage_similarity'] = df.babbage_similarity.apply(eval).apply(np.array)

In [40]:
df['babbage_similarity'][0]

array([-0.00644435,  0.01317739, -0.01283282, ..., -0.00715676,
       -0.02316985, -0.01435078])

In [1]:
df['babbage_similarity'][0].size

NameError: name 'df' is not defined

In [38]:
from sklearn.model_selection import train_test_split
 
X_train, X_test, y_train, y_test = train_test_split(
    list(df.babbage_similarity.values),
    df.Sentiment,
    test_size = 0.2,
    random_state=42
)

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
 
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)